<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/master/landlab_header.png"></a>


# Using the Landlab BiotaEvolver component

<hr>
<small> For instructions on how to run an interactive iPython notebook, click here: <a href="https://github.com/landlab/tutorials/blob/master/README.md">https://github.com/landlab/tutorials/blob/master/README.md</a></small><br>
<small>For more Landlab tutorials, click here: <a href="https://github.com/landlab/landlab/wiki/Tutorials">https://github.com/landlab/landlab/wiki/Tutorials</a></small>
<hr>

In this tutorial we will:
* Introduce species into a Landlab model.
* Set the zones in which the species operate.
* Evolve a landscape and the species over time.
* Explore BiotaEvolver output data structures.

The configuration of the model in this tutorial loosely follows the fault throw experiment in Lyons et al., in preperation for Earth Surface Dynamics.

Import modules and set the environment to show plots inline in this notebook.

In [2]:
from landlab import RasterModelGrid, CLOSED_BOUNDARY, FIXED_VALUE_BOUNDARY
from landlab.components import (BiotaEvolver, FastscapeEroder, FlowRouter,
                                LinearDiffuser)
from landlab.components.biota_macroevolution import Species, Zone
import numpy as np

%matplotlib inline

## Create a model grid with steady state topography

Here the topography of a Landlab model grid brought to topographic steady state.

Create a grid with random initial topography.

In [3]:
dx = 200
nrows = 100
ncols = 20
mg_ss = RasterModelGrid(nrows, ncols, dx)
mg_ss.set_status_at_node_on_edges(right=CLOSED_BOUNDARY,
                                  top=FIXED_VALUE_BOUNDARY,
                                  left=CLOSED_BOUNDARY,
                                  bottom=FIXED_VALUE_BOUNDARY)
z_ss = mg_ss.add_zeros('node', 'topographic__elevation')
np.random.seed(500)
z_ss += np.random.rand(z_ss.size)

Set parameters and initialize components.

In [ ]:
dt = 1000
total_time = int(4e6)
t = range(0, total_time, dt)
U = 7e-5
uplift_per_step = U * dt
K_sp = 10e-5
m_sp = 0.5
n_sp = 1
k_d = 0.0001

fr = FlowRouter(mg_ss)
sp = FastscapeEroder(mg_ss, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp)
lf = LinearDiffuser(mg_ss, linear_diffusivity=k_d, deposit=False)

Run the model until topography reaches steady state (ss).

In [ ]:
for t_i in t:
    z_ss[mg_ss.core_nodes] += uplift_per_step

    fr.run_one_step()
    sp.run_one_step(dt)
    lf.run_one_step(dt)

## Fault the grid and evolve species

Create a copy of the steady state grid.

In [ ]:
mg = deepcopy(mg_ss)
z = mg.at_node['topographic__elevation']

Define the fault.

In [ ]:
grid_width = mg.number_of_node_columns * mg.dx
x_of_fault = grid_width * 0.5
core_mask = mg.node_is_core()
upthown_block_mask = mg.x_of_node >= x_of_fault
upthown_block_mask = np.all([core_mask, upthown_block_mask], 0)
z[upthown_block_mask] += 100

Initialize components.

In [ ]:
fr = FlowRouter(mg)
sp = FastscapeEroder(mg, K_sp=K_sp, m_sp=m_sp, n_sp=n_sp)
lf = LinearDiffuser(mg, linear_diffusivity=k_d, deposit=False)

be = BiotaEvolver(mg)
dt_be = 1e5

Populate BiotaEvolver zones with species.

Zones are portions of a model grid. A Species object exists throughout a zone. At each timestep, the spatial intersection of the zones at the prior time and current time are identified and typed. For example, a zone in the prior time intersects two zones in the current time. This zone path is typed one-to-many.

Macroevolutionary processes are carried out by the intersection of zones over time. The connectivity of zones in BiotaEvolver are called 'paths', and the relationship of zones is described by the path type. Not all combinations of none, one, and many are considered at the time of this tutorial. All none-to-n and some many-to-n are not considered. 

<img style="float: left;" src="images/zone__one_to_none.png"> &nbsp; zone<sub>*t*</sub>: a zone in time, *t*.<br>

<img style="float: left;" src="images/zone__later_time.png"> &nbsp; zone<sub>*t*+1</sub>: a zone in the time following *t*.

<center>path type</center> | <center>graphical<br>represention</center> | <center>zone temporal connectivity</center> | <center>macroevolution implications</center>
--- | --- | --- | ---
one-to-none | ![](images/zone__one_to_none.png) | zone<sub>*t*</sub> does not intersect a zone in *t*+1. | The species in the zone of the earlier time will go extinct. 
one-to-one | ![](images/zone__one_to_one.png) | zone<sub>*t*</sub> intersects zone<sub>*t*+1</sub>. | The species in the zone of the earlier time will relocate to the zone of the later time.
one-to-many | ![](images/zone__one_to_many.png) | zone<sub>*t*</sub> intersects multiple zones in *t*+1. | The species in the zone of the earlier time disperses across the multiple zones in the later time. The species in the zones of the later time are geographical disconnected, thus speciation occurs.
many-to-one | ![](images/zone__many_to_one.png) | Multiple zones in *t* intersect zone<sub>*t*+1</sub>. | Species density increases locally.
many-to-many | ![](images/zone__many_to_many.png) | Multiple zones in *t* intersect multiple zones in *t*+1. | Species distribution shifts. Speciation occurs where the zone of a species at *t* intersects more than one zone at *t*+1.


In [1]:
min_stream_area = 1e6
zones = Zone.get_zones_with_area_threshold(mg, min_stream_area)
for zone in zones:
    new_species = Species(0, zone)
    be.introduce_species(new_species, 0)

NameError: name 'Zone' is not defined

Run model.

In [ ]:
total_time = 5e5
nt = int(total_time // dt)
for time_i in range(nt):
    # Uniformly uplift.
    z[mg.core_nodes] += uplift_per_step

    # Increment components.
    fr.run_one_step()
    sp.run_one_step(dt)
    lf.run_one_step(dt)

    elapsed_time = dt * (time_i + 1)

    if elapsed_time % dt_be == 0:
        # Run BiotaEvolver with updated zones.
        print('BiotaEvolver time', elapsed_time, '---------------------------')
        zones = Zone.get_zones_with_area_threshold(mg, min_stream_area)
        be.run_one_step(elapsed_time, zones)

## Species identifiers and phylogenetic trees

Species identifiers are automatically generated. They are stored as tuples in species objects. The first element of the identifier tuple is the clade name, and the second element is the species number. For example, the first species of clade A is 0, so the identifier is (A, 0) and the tenth species of this clade is (A, 9).

### Clade name

A clade is a group of species that share a common ancestor. Species may share multiple common ancestors, so clades can be defined differently in the same tree. The clade(s) of this example tree can be defined in multiple ways:

![](images/clade_definitions.png)

In BiotaEvolver, clades are defined by the root node for the purpose of assigning species identifiers. The species at the root node is:
* the least recent common ancestor of all species in the tree, and
* typically introduced to a BiotaEvolver model using the ```introduce_species``` function.

The clades of the first 26 species are labeled alphabetically from A to Z. The next 26 are labeled AB thru AZ. 

1-26: A to Z
27-53: AA to AZ
BA to BZ
...
AAA to AAZ
ABA to ABZ
ACA to ACZ

### Species number

Species are numbered sequently by clade beginning with 0.

In the example tree below, species A0 produced 4 child species. Only one species, B0 exists in clade B because child species were not produced by B0.

![](images/species_number.png)

### Click here for more <a href="https://github.com/landlab/landlab/wiki/Tutorials">Landlab tutorials</a>